In [1]:
import yaml
import platform
import pandas as pd
import glob
from PIL import Image

In [2]:
def check_os():
    os = platform.system()

    if os == 'Darwin':
        return "MacOS"
    elif os == 'Linux':
        return "Linux"
    else:
        return "Unknown OS"
    
operating_system = check_os()


if operating_system == "MacOS":
    root_path = "/Users/johnny/Projects/"
elif operating_system == "Linux":
    root_path = "/home/johnny/Projects/"

In [3]:
import os
import re

images_directory = '/data-fast/108-data3/ierregue/datasets/custom_dataset_v1/images/val'
labels_directory = '/data-fast/108-data3/ierregue/datasets/custom_dataset_v1/labels/val'

image_files = glob.glob(images_directory + '*.jpg')
# List to hold the DataFrame rows before concatenation
df_rows = []

# Function to get image resolution
def get_image_resolution(image_path):
    with Image.open(image_path) as img:
        return img.size

# Read and process each label file and associate it with the corresponding image file
for image_file in image_files:
    # Get the resolution of the image
    image_resolution = get_image_resolution(image_file)

    # Get the base of the file name to match with label files
    base_name = os.path.basename(image_file).replace('.jpg', '')
    
    # Construct the corresponding label file path
    label_file = os.path.join(labels_directory, base_name + '.txt')
    
    # Check if the label file exists
    if os.path.exists(label_file):
        # Read the annotation data from the label file
        with open(label_file, 'r') as file:
            annotation_data = file.readlines()
        
        # Process each annotation in the label file
        for line in annotation_data:
            class_id, x_center, y_center, width, height = line.strip().split()
            # Calculate object size in pixels
            object_width = int(float(width) * image_resolution[0])
            object_height = int(float(height) * image_resolution[1])

            df_rows.append({
                'file_name': base_name + '.jpg',
                'class_id': int(class_id),
                'x_center': float(x_center),
                'y_center': float(y_center),
                'width': float(width),
                'height': float(height),
                'res_width': image_resolution[0],
                'res_height': image_resolution[1],
                'obj_width': object_width,
                'obj_height': object_height,
                'image_path': image_file,
                'label_path': label_file,
            })

# Create a DataFrame from the list of rows
df_annotations = pd.DataFrame(df_rows)

# Display the DataFrame
print(df_annotations.head())

NameError: name 'os' is not defined

In [ ]:
df_annotations.to_csv('/data-fast/108-data3/ierregue/datasets/custom_dataset_v1/annotations_valid.csv', index=False)

In [ ]:
full_hd_new_data = df_annotations[(df_annotations['res_width'] == 1920) & (df_annotations['res_height'] == 1080)].copy()